In [3]:
import os.path as p
import pandas as pd

from sentence_transformers import SentenceTransformer
from definitions import *
from dataset_helper_functions import *

Load devset.

In [7]:
try:
    IS_MASTER
except: 
    IS_MASTER = False

if not IS_MASTER:
    # train_path = p.join(POLIT_dev_DIR_PATH, 'train')
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')
    dev_tsv_path = p.join(dev_path, 'dev.tsv')
    dev_pkl_path = p.join(dev_path, 'dev_spacy.pkl')

    if p.exists(dev_pkl_path):
        dev = pd.read_pickle(dev_pkl_path)
    else:
        if not p.exists(dev_tsv_path):
            sample_development_set()

        dev = pd.read_csv(dev_tsv_path, sep='\t', index_col=False)

In [31]:
embeddings_path = p.join(PROC_DATA_DIR_PATH, 'embeddings')
if not p.exists(embeddings_path):
    os.mkdir(embeddings_path)

Load model.

3 models to try:
- `all-mpnet-base-v2`: best performance overall, slow
- `multi-qa-mpnet-base-dot-v1`: tuned for semantic search, good performance, slow 
- `all-MiniLM-L6-v2`: smallest with moderately good performance, fast; **dev done with this due to it's speed**

In [11]:
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

Get sentences from devset.

Models accept list of sentences to encode.

In [14]:
sentences = dev['content'].values
embeddings = model.encode(sentences)

(654, 384)
(384,)


Create devframe with sentences ids and embeddings and save it.

In [37]:
embeddings_df = dev.loc[:, ['id']].merge(pd.DataFrame(embeddings), left_index=True, right_index=True)

embeddings_df.to_csv(p.join(embeddings_path, f'dev_emb_{model_name}.tsv'), sep='\t', index=False)